In [2]:
import scrape_trustpilot_reviews
import scrape_google_reviews
import pandas as pd
from datetime import datetime
import sys

In [ ]:
print("Starting Google Reviews scraping process...")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


# Step 1: Get establishments to scrape
print("\nStep 1: Getting establishments to scrape...")
establishments = scrape_google_reviews.get_establishments_to_scrape()

if establishments.empty:
    print("No establishments found matching the criteria.")
else:
    print(f"Found {len(establishments)} establishments to scrape:")
    display(establishments[['placeId', 'title', 'website', 'reviewsCount']])
    
    # Step 2: Scrape reviews
    print("\nStep 2: Scraping reviews...")
    reviews_df = scrape_google_reviews.scrape_reviews(establishments)
    print(f"Scraped {len(reviews_df)} reviews")
    if not reviews_df.empty:
        print("\nFirst few reviews:")
        display(reviews_df.head())
    
    # Step 3: Save individual review files
    print("\nStep 3: Saving individual review files...")
    scrape_google_reviews.save_reviews(reviews_df)
    


In [6]:
# Step 4: Unify reviews
print("\nStep 4: Unifying reviews...")
scrape_google_reviews.unify_reviews()

# Step 5: Update establishment base
print("\nStep 5: Updating establishment base...")
scrape_google_reviews.update_establishment_base()

print("\nProcess completed successfully!")
print(f"End timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")



Step 4: Unifying reviews...
Found existing unified file: reviews\google\allGoogleReviews_2025-04-05.xlsx
Previously processed files: 3
Processing new file: reviews\google\googleReviews_2025-04-04_23-13.xlsx
Processed 1 new files

Saved unified reviews to: reviews/google/allGoogleReviews_2025-04-05.xlsx

Step 5: Updating establishment base...

Process completed successfully!
End timestamp: 2025-04-05 09:17:13


In [1]:
import pandas as pd
import hashlib
from pathlib import Path
import os

def generate_review_id(place_id, published_at):
    return hashlib.md5(f"{place_id}_{published_at}".encode()).hexdigest()

# Function to process a single file
def process_file(file_path):
    print(f"Processing {file_path}")
    df = pd.read_excel(file_path)
    
    # Generate reviewId for each row
    df['reviewId'] = df.apply(
        lambda row: generate_review_id(row['placeId'], row['publishedAtDate']), 
        axis=1
    )
    
    # Save the updated file
    df.to_excel(file_path, index=False)
    print(f"Updated {file_path}")

# Process all Google review files
google_dir = Path("reviews/google")
for file in google_dir.glob("googleReviews_*.xlsx"):
    if not file.name.startswith("allGoogleReviews_"):  # Skip unified files
        process_file(file)

# Process the unified file if it exists
unified_files = list(google_dir.glob("allGoogleReviews_*.xlsx"))
if unified_files:
    latest_unified = max(unified_files)
    process_file(latest_unified)

print("All files have been processed!")

Processing reviews\google\googleReviews_2025-04-03_13-30.xlsx
Updated reviews\google\googleReviews_2025-04-03_13-30.xlsx
Processing reviews\google\googleReviews_2025-04-03_18-56.xlsx
Updated reviews\google\googleReviews_2025-04-03_18-56.xlsx
Processing reviews\google\googleReviews_2025-04-04_23-13.xlsx
Updated reviews\google\googleReviews_2025-04-04_23-13.xlsx
Processing reviews\google\googleReviews_2025-04-04_23-25.xlsx
Updated reviews\google\googleReviews_2025-04-04_23-25.xlsx
Processing reviews\google\allGoogleReviews_2025-04-05.xlsx
Updated reviews\google\allGoogleReviews_2025-04-05.xlsx
All files have been processed!


In [4]:
import pandas as pd
from pathlib import Path
import os

# Function to analyze duplicates
def analyze_duplicates():
    google_dir = Path("reviews/google")
    all_reviews = []
    
    # Read all individual review files
    for file in google_dir.glob("googleReviews_*.xlsx"):
        if not file.name.startswith("allGoogleReviews_"):  # Skip unified files
            print(f"Reading {file.name}")
            df = pd.read_excel(file)
            df['source_file'] = file.name
            all_reviews.append(df)
    
    if not all_reviews:
        print("No review files found!")
        return
    
    # Combine all reviews
    combined_df = pd.concat(all_reviews, ignore_index=True)
    
    # Count total reviews
    total_reviews = len(combined_df)
    print(f"\nTotal number of reviews across all files: {total_reviews}")
    
    # Count unique reviews
    unique_reviews = combined_df['reviewId'].nunique()
    print(f"Number of unique reviews: {unique_reviews}")
    
    # Calculate duplicates
    duplicates = total_reviews - unique_reviews
    print(f"Number of duplicate reviews: {duplicates}")
    
    if duplicates > 0:
        print("\nAnalyzing duplicate patterns:")
        # Find reviews that appear in multiple files
        duplicate_reviews = combined_df[combined_df.duplicated(subset=['reviewId'], keep=False)]
        
        # Group by reviewId and count occurrences
        duplicate_counts = duplicate_reviews.groupby('reviewId').size().reset_index(name='occurrences')
        duplicate_counts = duplicate_counts[duplicate_counts['occurrences'] > 1]
        
        print(f"\nReviews appearing in multiple files:")
        print(duplicate_counts['occurrences'].value_counts().sort_index())
        
        # Show example of a duplicate review
        print("\nExample of a duplicate review:")
        example_review_id = duplicate_counts.iloc[0]['reviewId']
        example_review = combined_df[combined_df['reviewId'] == example_review_id]
        print(example_review[['reviewId', 'text', 'publishedAtDate', 'source_file']])
    
    # Check for reviews with same reviewId but different content
    print("\nChecking for reviews with same reviewId but different content:")
    review_groups = combined_df.groupby('reviewId')
    for review_id, group in review_groups:
        if len(group) > 1:
            # Check if any of these columns differ
            diff_columns = ['text', 'stars', 'likesCount', 'responseFromOwnerText']
            if group[diff_columns].nunique().any():
                print(f"\nReview {review_id} has different content across files:")
                print(group[['source_file'] + diff_columns])
                break  # Just show the first example

# Run the analysis
analyze_duplicates()

Reading googleReviews_2025-04-03_13-30.xlsx
Reading googleReviews_2025-04-03_18-56.xlsx
Reading googleReviews_2025-04-04_23-13.xlsx
Reading googleReviews_2025-04-04_23-25.xlsx

Total number of reviews across all files: 8467
Number of unique reviews: 6663
Number of duplicate reviews: 1804

Analyzing duplicate patterns:

Reviews appearing in multiple files:
occurrences
2     740
3     121
4      52
5      16
6      19
7       7
8       7
9       3
10      3
11      1
12      1
13      5
14      3
15      2
16      1
18      1
20      1
21      1
23      1
29      1
30      1
68      1
Name: count, dtype: int64

Example of a duplicate review:
                              reviewId  \
6774  00025b35461fc3a8f5b86bf061189b8a   
6775  00025b35461fc3a8f5b86bf061189b8a   

                                                   text      publishedAtDate  \
6774  Hi , I m kium\nI heard about hair transplant l...  2021-08-13 00:00:00   
6775  My experience is owesome from the day i land i...  2021-08-

In [5]:
import pandas as pd
from pathlib import Path
import os

def analyze_unified_file():
    google_dir = Path("reviews/google")
    
    # Find the latest unified file
    unified_files = list(google_dir.glob("allGoogleReviews_*.xlsx"))
    if not unified_files:
        print("No unified file found!")
        return
    
    latest_unified = max(unified_files)
    print(f"Analyzing unified file: {latest_unified.name}")
    
    # Read the unified file
    unified_df = pd.read_excel(latest_unified)
    unified_review_ids = set(unified_df['reviewId'])
    print(f"Number of reviews in unified file: {len(unified_review_ids)}")
    
    # Read all individual files
    all_reviews = []
    for file in google_dir.glob("googleReviews_*.xlsx"):
        if not file.name.startswith("allGoogleReviews_"):  # Skip unified files
            print(f"\nReading {file.name}")
            df = pd.read_excel(file)
            df['source_file'] = file.name
            all_reviews.append(df)
            print(f"Number of reviews in {file.name}: {len(df)}")
    
    if not all_reviews:
        print("No individual review files found!")
        return
    
    # Combine all individual reviews
    combined_df = pd.concat(all_reviews, ignore_index=True)
    all_review_ids = set(combined_df['reviewId'])
    print(f"\nTotal number of reviews across all individual files: {len(all_review_ids)}")
    
    # Find missing reviews
    missing_reviews = all_review_ids - unified_review_ids
    print(f"\nNumber of reviews missing from unified file: {len(missing_reviews)}")
    
    if missing_reviews:
        print("\nExample of missing reviews:")
        missing_df = combined_df[combined_df['reviewId'].isin(missing_reviews)]
        print(missing_df[['reviewId', 'text', 'publishedAtDate', 'source_file']].head())
        
        # Analyze which files contain the missing reviews
        print("\nFiles containing missing reviews:")
        missing_files = missing_df['source_file'].value_counts()
        print(missing_files)

# Run the analysis
analyze_unified_file()

Analyzing unified file: allGoogleReviews_2025-04-05.xlsx
Number of reviews in unified file: 2922

Reading googleReviews_2025-04-03_13-30.xlsx
Number of reviews in googleReviews_2025-04-03_13-30.xlsx: 496

Reading googleReviews_2025-04-03_18-56.xlsx
Number of reviews in googleReviews_2025-04-03_18-56.xlsx: 344

Reading googleReviews_2025-04-04_23-13.xlsx
Number of reviews in googleReviews_2025-04-04_23-13.xlsx: 3742

Reading googleReviews_2025-04-04_23-25.xlsx
Number of reviews in googleReviews_2025-04-04_23-25.xlsx: 3885

Total number of reviews across all individual files: 6663

Number of reviews missing from unified file: 3741

Example of missing reviews:
                             reviewId  \
840  3600819d589df7232a217a151d6ff9a6   
841  3a31a8e89c49295f47dc40e802b9c9ee   
842  188f31136d94b4e604330789cfbf6d66   
843  35e1ee2e662dfb3210620e592191b1d8   
844  0eab2b332674d42b6c8a903ed1010bcd   

                                                  text  \
840  I had my hair transplant